In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import numpy as np

In [2]:
# The Portfolio
hisse_bilgileri = {
    'ASTOR': {'Maaliyet': 101.831, 'Portföy Lot': 589},
    'MIATK': {'Maaliyet': 14.982, 'Portföy Lot': 766},
    'YUNSA': {'Maaliyet': 71.472, 'Portföy Lot': 439.47},
    'GWIND': {'Maaliyet': 22.593, 'Portföy Lot': 1021.12},
    'TTRAK': {'Maaliyet': 824.630, 'Portföy Lot': 30},
    'FROTO': {'Maaliyet': 1038, 'Portföy Lot': 5},
}

fon_bilgileri = {
    'CPU': {'Maaliyet': 1.431, 'Portföy Lot': 12921},
    'KPC': {'Maaliyet': 12.86 , 'Portföy Lot': 777}
}

# Portfolio Weights
hisse_türü = {
    'ATAK': {'Tür %': 35},
    'ORTA': {'Tür %': 25},
    'DEFANS': {'Tür %': 20},
}

fon_türü ={
    'FON/NASDAQ': {'Tür %': 10},
    'FON/BIST': {'Tür %': 10},
}

# Define target stocks and funds
hedef_hisseler = ['GWIND', 'FROTO', 'MIATK', 'YUNSA', 'ASTOR', 'TTRAK']
hedef_fonlar = ['CPU', 'KPC']

# Cash/TL
tl = 111

In [3]:
def clean_price(price):
    if pd.isna(price):
        return None
    price = price.replace('.', '', price.count('.') - 1)  # Son iki nokta ondalık ayırıcı olarak kalır
    return price.replace(',', '.')

def clean_percentage(percentage):
    if pd.isna(percentage) or not isinstance(percentage, str):
        return percentage
    # % işaretini ve binlik ayırıcıları kaldır
    return percentage.replace('%', '').replace(',', '.')

In [4]:
def get_stock_data():
    try:
        url = 'https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/default.aspx'
        response = requests.get(url)

    except Exception as e:
        print(e)
        print('No response from the website')
        return None

    if response is None:
        print('No response received')
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    hisse_fiyatlari_tablosu = soup.find('div', class_='single-table')

    data = []

    for row in hisse_fiyatlari_tablosu.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) > 1:
            hisse_adi = cols[0].text.strip()
            fiyat = cols[1].text.strip().replace(',', '.')
            degisim = cols[2].text.strip().replace(',', '.')
            data.append([hisse_adi, fiyat, degisim])

    df = pd.DataFrame(data, columns=['Hisse/Fon Adı', 'Fiyat', 'Değişim (%)'])
    return df

In [5]:
# Calculate portfolio percentages
def get_stock_group(stock_name):
    if stock_name in ['ASTOR', 'MIATK']:
        return 'ATAK'
    elif stock_name in ['GWIND', 'YUNSA']:
        return 'ORTA'
    elif stock_name in ['TTRAK', 'FROTO']:
        return 'DEFANS'
    else:
        return None  # Returns None if the provided stock name doesn't match any defined group

In [6]:
def get_fund_price(fund_code, retries=3, delay=5):
    url = f'https://www.tefas.gov.tr/FonAnaliz.aspx?FonKod={fund_code}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            price = None
            daily_return = None
            
            # Find the <li> tag that contains the "Son Fiyat (TL)" heading
            li_tags = soup.find_all('li')
            for li in li_tags:
                if 'Son Fiyat (TL)' in li.text:
                    # Find the <span> tag inside the <li> tag
                    price_span = li.find('span')
                    if price_span:
                        price = price_span.text.strip().replace(',', '.')
                if 'Günlük Getiri (%)' in li.text:
                    # Find the <span> tag inside the <li> tag
                    daily_span = li.find('span')
                    if daily_span:
                        daily_return = daily_span.text.strip().replace(',', '.').replace('%', '')
                        daily_return = round(float(daily_return), 2)
                        
            if price is not None and daily_return is not None:
                return price, daily_return
            else:
                print(f"Price or daily return not found for fund code {fund_code}, retrying...")
                time.sleep(delay)  # Wait before retrying

        except requests.exceptions.RequestException as e:
            print(f"An error occurred on attempt {attempt + 1}: {e}")
            time.sleep(delay)  # Wait before retrying

    return None, None

def fetch_fund_price_until_success(fund_code):
    while True:
        price, daily_return = get_fund_price(fund_code)
        if price is not None and daily_return is not None:
            return price, daily_return
        print(f"Retrying to fetch the price and daily return for fund code {fund_code}...")

In [7]:
# Fetch data and create a DataFrame
df = get_stock_data()

# Data cleaning and formatting
df['Hisse/Fon Adı'] = df['Hisse/Fon Adı'].str.replace(r'\r|\n|\u200b', '', regex=True)
df['Hisse/Fon Adı'] = df['Hisse/Fon Adı'].astype(str)

# Sort by stock names
df = df.sort_values(by='Hisse/Fon Adı', ascending=True)

# Remove single quotes
df['Hisse/Fon Adı'] = [x.split()[0].strip("'") for x in df['Hisse/Fon Adı']]

# Filter for target stocks
df_filtered = df[df['Hisse/Fon Adı'].isin(hedef_hisseler)].copy()

In [8]:
df_filtered

,Hisse/Fon Adı,Fiyat,Değişim (%)
488,ASTOR,98.30,0.46
102,FROTO,1.002.00,0.30
388,GWIND,33.58,3.96
432,MIATK,61.20,-3.62
78,TTRAK,809.00,2.08
16,YUNSA,79.50,-0.93


In [9]:
# Fetch fund prices and daily returns
fon_fiyatlar = []
fon_getiriler = []
for fon in hedef_fonlar:
    fiyat, getiri = fetch_fund_price_until_success(fon)
    fon_fiyatlar.append(fiyat)
    fon_getiriler.append(getiri)

# Add target funds to df_filtered
df_fonlar = pd.DataFrame({
    'Hisse/Fon Adı': hedef_fonlar,
    'Fiyat': fon_fiyatlar,
    'Değişim (%)': fon_getiriler
})

# Combine stock and fund data
df_filtered = pd.concat([df_filtered, df_fonlar], ignore_index=True)

# Ensure that 'Hisse/Fon Adı' columns are used for matching
df_filtered_sorted = df_filtered.sort_values(by='Hisse/Fon Adı').reset_index(drop=True)

# Portfolio data
maaliyet_list = [
    hisse_bilgileri[hisse]['Maaliyet'] for hisse in df_filtered_sorted['Hisse/Fon Adı'] if hisse in hisse_bilgileri
] + [
    fon_bilgileri[fon]['Maaliyet'] for fon in df_filtered_sorted['Hisse/Fon Adı'] if fon in fon_bilgileri
]

portfoy_lot_list = [
    hisse_bilgileri[hisse]['Portföy Lot'] for hisse in df_filtered_sorted['Hisse/Fon Adı'] if hisse in hisse_bilgileri
] + [
    fon_bilgileri[fon]['Portföy Lot'] for fon in df_filtered_sorted['Hisse/Fon Adı'] if fon in fon_bilgileri
]

Price or daily return not found for fund code CPU, retrying...
Price or daily return not found for fund code CPU, retrying...
Price or daily return not found for fund code CPU, retrying...
Retrying to fetch the price and daily return for fund code CPU...
Price or daily return not found for fund code CPU, retrying...
Price or daily return not found for fund code KPC, retrying...
Price or daily return not found for fund code KPC, retrying...


In [10]:
df_filtered

,Hisse/Fon Adı,Fiyat,Değişim (%)
0,ASTOR,98.30,0.46
1,FROTO,1.002.00,0.30
2,GWIND,33.58,3.96
3,MIATK,61.20,-3.62
4,TTRAK,809.00,2.08
5,YUNSA,79.50,-0.93
6,CPU,1.436981,-2.34
7,KPC,12.591706,-0.89


In [11]:
maaliyet_list

[101.831, 1038, 22.593, 14.982, 824.63, 71.472, 1.431, 12.86]

In [12]:
portfoy_lot_list

[589, 5, 1021.12, 766, 30, 439.47, 12921, 777]

In [13]:
df_filtered['Maaliyet'] = maaliyet_list
df_filtered['Portföy Lot'] = portfoy_lot_list

In [14]:
df_filtered

,Hisse/Fon Adı,Fiyat,Değişim (%),Maaliyet,Portföy Lot
0,ASTOR,98.30,0.46,101.831,589.00
1,FROTO,1.002.00,0.30,1038.000,5.00
2,GWIND,33.58,3.96,22.593,1021.12
3,MIATK,61.20,-3.62,14.982,766.00
4,TTRAK,809.00,2.08,824.630,30.00
5,YUNSA,79.50,-0.93,71.472,439.47
6,CPU,1.436981,-2.34,1.431,12921.00
7,KPC,12.591706,-0.89,12.860,777.00


In [15]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Hisse/Fon Adı  8 non-null      object 
 1   Fiyat          8 non-null      object 
 2   Değişim (%)    8 non-null      object 
 3   Maaliyet       8 non-null      float64
 4   Portföy Lot    8 non-null      float64
dtypes: float64(2), object(3)
memory usage: 452.0+ bytes


In [16]:
# Fiyat ve Değişim (%) sütunlarını dönüştürme
df_filtered['Fiyat'] = df_filtered['Fiyat'].apply(lambda x: clean_price(x))
df_filtered['Fiyat'] = pd.to_numeric(df_filtered['Fiyat'], errors='coerce')

df_filtered['Değişim (%)'] = df_filtered['Değişim (%)'].apply(lambda x: clean_percentage(x))
df_filtered['Değişim (%)'] = pd.to_numeric(df_filtered['Değişim (%)'], errors='coerce')

# Kontrol
print(df_filtered.dtypes)

Hisse/Fon Adı     object
Fiyat            float64
Değişim (%)      float64
Maaliyet         float64
Portföy Lot      float64
dtype: object


In [17]:
df_filtered

,Hisse/Fon Adı,Fiyat,Değişim (%),Maaliyet,Portföy Lot
0,ASTOR,98.300000,0.46,101.831,589.00
1,FROTO,1002.000000,0.30,1038.000,5.00
2,GWIND,33.580000,3.96,22.593,1021.12
3,MIATK,61.200000,-3.62,14.982,766.00
4,TTRAK,809.000000,2.08,824.630,30.00
5,YUNSA,79.500000,-0.93,71.472,439.47
6,CPU,1.436981,-2.34,1.431,12921.00
7,KPC,12.591706,-0.89,12.860,777.00


In [ ]:
df_filtered['Güncel Tutar'] = df_filtered['Fiyat'] * df_filtered['Portföy Lot']
"""
df_filtered['₺ KAR/ZARAR '] = df_filtered['Güncel Tutar'] - (df_filtered['Maaliyet'] * df_filtered['Portföy Lot'])
df_filtered['Hisse Türü'] = df_filtered['Hisse Adı'].apply(get_stock_group)
df_filtered['% Portföy Oranı'] = ((df_filtered['Güncel Tutar'].fillna(0) / Toplam_portföy) * 100).round(2)
df_filtered['%KAR/ZARAR'] = ((df_filtered['Fiyat'] - df_filtered['Maaliyet']) / df_filtered['Maaliyet'] * 100).round(2)
"""

In [ ]:




# Calculate total portfolio (including cash amount)
Toplam_portföy = sum(df_filtered['Güncel Tutar'].fillna(0)) + tl

# Create the 'Hisse Türü' column and fill it with stock groups



In [ ]:
df_filtered

In [ ]:
# Dictionary to store the count of each stock type
stock_type_counts = {
    'ATAK': len(df_filtered[df_filtered['Hisse Türü'] == 'ATAK']),
    'ORTA': len(df_filtered[df_filtered['Hisse Türü'] == 'ORTA']),
    'DEFANS': len(df_filtered[df_filtered['Hisse Türü'] == 'DEFANS'])
}

# Calculate 'DELTA %' based on stock type counts
def calculate_delta(row):
    stock_type = row['Hisse Türü']
    if stock_type == 'ATAK':
        return row['% Portföy Oranı'] - (55 / stock_type_counts['ATAK'])
    elif stock_type == 'ORTA':
        return row['% Portföy Oranı'] - (35 / stock_type_counts['ORTA'])
    elif stock_type == 'DEFANS':
        return row['% Portföy Oranı'] - (10 / stock_type_counts['DEFANS'])
    else:
        return None

# Apply the calculation to create the 'DELTA %' column
df_filtered['DELTA %'] = df_filtered.apply(calculate_delta, axis=1)

# Calculate 'DELTA ₺' column correctly
def calculate_delta_tl(row):
    stock_type = row['Hisse Türü']
    stock_type_count = stock_type_counts.get(stock_type)
    if stock_type_count is None:
        return None

    if stock_type == 'ATAK':
        return row['Güncel Tutar'] - (Toplam_portföy * (0.55 / stock_type_count))
    elif stock_type == 'ORTA':
        return row['Güncel Tutar'] - (Toplam_portföy * (0.35 / stock_type_count))
    elif stock_type == 'DEFANS':
        return row['Güncel Tutar'] - (Toplam_portföy * (0.10 / stock_type_count))
    else:
        return None


# Apply the calculation to create the 'DELTA ₺' column
df_filtered['DELTA ₺'] = df_filtered.apply(calculate_delta_tl, axis=1)

df_filtered['ÇARPAN'] = round(df_filtered['DELTA ₺']/1000,2)

df_filtered = df_filtered.drop(['Hacim(TL)', 'Hacim(Adet)'], axis=1)
# Print the total portfolio value and the filtered DataFrame
print("Toplam portföy : {}".format(Toplam_portföy))
print(df_filtered)

In [ ]:
# Define target stocks and funds
hedef_hisseler = ['GWIND', 'FROTO', 'MIATK', 'YUNSA', 'ASTOR', 'TTRAK']
hedef_fonlar = ['CPU', 'KPC']

# Fetch stock and fund data
df_stocks = get_stock_data()  # Assuming this returns stocks data with 'Hisse Adı' column
df_funds = pd.DataFrame({
    'Hisse/Fon Adı': hedef_fonlar,
    'Fiyat': [get_fund_price(fund) for fund in hedef_fonlar]
})

# Process stock data
df_stocks['Hisse Adı'] = df_stocks['Hisse Adı'].astype(str)
df_stocks = df_stocks[df_stocks['Hisse Adı'].isin(hedef_hisseler)].copy()
df_stocks['Fiyat'] = df_stocks['Fiyat'].str.replace(',', '.').astype(float)
df_stocks['Hisse/Fon Adı'] = df_stocks['Hisse Adı']  # Rename column for merging

# Combine stocks and funds data
df_combined = pd.concat([df_stocks[['Hisse/Fon Adı', 'Fiyat']], df_funds], ignore_index=True)

# Print the combined DataFrame
print(df_combined)